In [195]:
from random import uniform
from math import cos, sin, pi
import pandas as pd
import numpy as np
import sklearn as sl

In [196]:
def generate1():  
    a = uniform(0, 1)  
    b = uniform(0, 1)  
    return (a * cos(2 * pi * b), a * sin(2 * pi * b))

def generate2():  
    while True:  
        x = uniform(-1, 1)  
        y = uniform(-1, 1)  
        if x ** 2 + y ** 2 > 1:  
            continue  
        return (x, y)

In [197]:
df1 = pd.DataFrame(columns=['x', 'y', 'def'])
df2 = pd.DataFrame(columns=['x', 'y', 'def'])

for i in range(10):
    x1, y1 = generate1()
    df1.loc[len(df1)] = [x1, y1, 1]
    
    x2, y2 = generate2()
    df2.loc[len(df2)] = [x2, y2, 2]

train = df1.append(df2, ignore_index=True).sample(frac=1)

XY_train = train.drop(['def'], axis=1)
DEF_train = train['def']

In [217]:
x1, y1 = generate1()
x2, y2 = generate2()
print(x1, y1, '\n', x2, y2)
print(len(str(x1)), len(str(y1)), len(str(x2)), len(str(y2)))

-0.11591654933961491 0.5761505690812128 
 0.5902032444798075 0.08439897010789865
20 18 18 19


In [203]:
df3 = pd.DataFrame(columns=['x', 'y', 'def'])
df4 = pd.DataFrame(columns=['x', 'y', 'def'])

for i in range(100):
    x3, y3 = generate1()
    df3.loc[len(df3)] = [x3, y3, 1]
    
    x4, y4 = generate2()
    df4.loc[len(df4)] = [x4, y4, 2]

test = df3.append(df4, ignore_index=True).sample(frac=1)

XY_test = test.drop(['def'], axis=1)
DEF_test = test['def']

In [191]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

In [192]:
rf = RandomForestClassifier()
rf.fit(XY_train, DEF_train)

rf.score(XY_train, DEF_train)

1.0

In [171]:
DEF_pred = rf.predict(XY_test)

precision_score(DEF_test, DEF_pred)

0.57

In [193]:
from statistics import mode 

while True:
    data = input()
    if data:
        data = data.split(' ')
        X_col = [int(data[i]) for i in range(0, len(data), 2)]
        Y_col = [int(data[i]) for i in range(1, len(data), 2)]

        test = pd.DataFrame(columns=['x', 'y'])

        for i in range(len(X_col)):
            test.loc[len(test)] = [X_col[i], Y_col[i]]
            
        pred = rf.predict(test)
        print(int(mode(pred)))

        
    else:
        break

1 2 3 4 5 6
2
2 3 5 4 6 4
2
0 0 0 0 0 0
1



In [176]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 3)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 30, num = 5)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 52, num = 5)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 27, num = 5)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(rf, 
                        param_dist, 
                        n_iter = 50, 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(XY_train, DEF_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.2min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 8, 15, 22, 30],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [2, 8, 14, 20, 27],
                                        'min_samples_split': [2, 14, 27, 39,
                                                              52],
                                        'n_estimators': [100, 200, 300]},
                   random_state=0, verbose=1)

In [177]:
DEF_pred_rs = rs.predict(XY_test)

precision_score(DEF_test, DEF_pred_rs)

0.581081081081081

In [172]:
from sklearn import tree

In [173]:
dtf = tree.DecisionTreeClassifier()
dtf.fit(XY_train, DEF_train)

dtf.score(XY_train, DEF_train)

1.0

In [174]:
DEF_pred = dtf.predict(XY_test)

precision_score(DEF_test, DEF_pred)

0.6037735849056604